In [59]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [60]:
load_dotenv()

True

In [61]:
model = ChatOpenAI()

In [62]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    evaluate: int | str | None

In [63]:
def create_outline(state: BlogState) -> BlogState:
    title = state['title']
    prompt = f"Create an outline for a blog post titled: {title}"
    
    outline = model.invoke(prompt).content
    state['outline'] = outline
    
    return state

In [64]:
def create_blog(state: BlogState) -> BlogState:
    outline = state['outline']
    prompt = f"Write a blog post based on the following outline:\n{outline}"
    
    content = model.invoke(prompt).content
    state['content'] = content
    
    return state

In [65]:
def evaluate_blog(state: BlogState) -> BlogState:
    content = state['outline']
    prompt = f"Base on this ouline: \n{content}\nRate it on a scale of 1 to 10."
    
    evaluation = model.invoke(prompt).content
    # Extract the first integer from the response
    import re
    match = re.search(r'\d+', evaluation)
    if match:
        state['evaluate'] = int(match.group(0))
    else:
        state['evaluate'] = None
    return state

In [66]:
graph = StateGraph(BlogState)

graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)
graph.add_node('evaluate_blog', evaluate_blog)


graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog') 
graph.add_edge('create_blog', 'evaluate_blog')
graph.add_edge('evaluate_blog', END)

workflow = graph.compile()

In [67]:
initial_state = {'title': 'The Future of AI in Everyday Life'}

final_state = workflow.invoke(initial_state)

print(final_state)  # Output should include the blog content based on the title

{'title': 'The Future of AI in Everyday Life', 'outline': "I. Introduction\n   A. Brief overview of artificial intelligence (AI) \n   B. Explanation of AI's increasing presence in everyday life \n\nII. Current applications of AI in everyday life\n   A. Virtual assistants like Siri and Alexa \n   B. Recommendations on streaming services and shopping websites \n   C. Smart home devices like thermostats and security systems \n\nIII. Future potential applications of AI in everyday life\n   A. Personalized healthcare and medical diagnostics \n   B. Autonomous vehicles and transportation systems \n   C. Enhanced education and learning experiences \n   D. AI-powered entertainment and content creation \n\nIV. Ethical considerations and challenges \n   A. Privacy concerns and data security \n   B. Bias and discrimination in AI algorithms \n   C. Impact on job displacement and workforce changes \n\nV. Opportunities for advancements and innovation \n   A. Collaboration between industries and rese

In [68]:
print(final_state['outline'])  # Print the generated blog content

I. Introduction
   A. Brief overview of artificial intelligence (AI) 
   B. Explanation of AI's increasing presence in everyday life 

II. Current applications of AI in everyday life
   A. Virtual assistants like Siri and Alexa 
   B. Recommendations on streaming services and shopping websites 
   C. Smart home devices like thermostats and security systems 

III. Future potential applications of AI in everyday life
   A. Personalized healthcare and medical diagnostics 
   B. Autonomous vehicles and transportation systems 
   C. Enhanced education and learning experiences 
   D. AI-powered entertainment and content creation 

IV. Ethical considerations and challenges 
   A. Privacy concerns and data security 
   B. Bias and discrimination in AI algorithms 
   C. Impact on job displacement and workforce changes 

V. Opportunities for advancements and innovation 
   A. Collaboration between industries and researchers 
   B. Investment in AI education and training 
   C. Regulation and pol

In [69]:
print(final_state['content'])  # Print the generated blog content

In today's world, artificial intelligence (AI) is no longer just a futuristic concept from science fiction movies. It has become an integral part of our everyday lives, with its presence in various applications and technologies continuing to grow. From virtual assistants like Siri and Alexa to personalized recommendations on streaming services and shopping websites, AI is revolutionizing the way we interact with technology and enhancing our day-to-day experiences.

The current applications of AI in everyday life are vast and diverse. Virtual assistants have become a staple in many households, providing quick answers to questions, setting reminders, and controlling smart home devices. Recommendations on streaming services and shopping websites use AI algorithms to analyze user data and preferences, offering personalized suggestions for entertainment and shopping. Smart home devices like thermostats and security systems use AI to learn and adapt to users' habits and behaviors, making hom

In [70]:
print(final_state['evaluate'])  # Print the generated blog content

8
